In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import rich
import requests
from io import BytesIO

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

## File Search is only part of Responses API

https://platform.openai.com/docs/guides/tools-file-search

In [ ]:
# Create function to Upload a file
def create_file(openai, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = openai.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = openai.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

file-8kai7amHc4aJ55jZ7Fx9xJ


In [ ]:
# Call function to upload a file

file_id = create_file(openai, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")
# file_id = create_file(openai, "docs/Panaversity-Certified-Agentic-and-Robotic-AI-Engineer.pdf")

In [4]:
# Create a vector store
vector_store = openai.vector_stores.create(
    name="knowledge_base_1"
)
print(vector_store.id)

vs_67dd115384c08191b70680374f0265b7


In [23]:
# Add a file to a vector store
result = openai.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
rich.print(result)

VectorStoreFile(
    id='file-8kai7amHc4aJ55jZ7Fx9xJ',
    created_at=1742552850,
    last_error=None,
    object='vector_store.file',
    status='in_progress',
    usage_bytes=0,
    vector_store_id='vs_67dd115384c08191b70680374f0265b7',
    attributes={},
    chunking_strategy=StaticFileChunkingStrategyObject(
        static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
        type='static'
    )
)

In [24]:
# Check status
result = openai.vector_stores.files.list(
    vector_store_id=vector_store.id
)
rich.print(result)

SyncCursorPage[VectorStoreFile](
    data=[
        VectorStoreFile(
            id='file-8kai7amHc4aJ55jZ7Fx9xJ',
            created_at=1742541160,
            last_error=None,
            object='vector_store.file',
            status='completed',
            usage_bytes=66539,
            vector_store_id='vs_67dd115384c08191b70680374f0265b7',
            attributes={},
            chunking_strategy=StaticFileChunkingStrategyObject(
                static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
                type='static'
            )
        )
    ],
    has_more=False,
    object='list',
    first_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
    last_id='file-8kai7amHc4aJ55jZ7Fx9xJ'
)

In [14]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output)

Deep Research by OpenAI is a new capability within ChatGPT aimed at performing extensive online research tasks with
a high level of autonomy. 

### Key Features:
- **Agentic Capability**: It can conduct multi-step research independently, synthesizing information from various 
sources to provide comprehensive and detailed reports.
- **Efficiency**: What would typically take a human numerous hours can be accomplished in a fraction of that 
time—often within minutes.
- **Advanced Reasoning**: Leveraging a version of the OpenAI o3 model optimized for web browsing and data analysis,
it reasons through large volumes of text, images, and documents online to create accurate outputs.
- **Documentation and Citations**: All outputs are fully documented with citations and detailed summaries, making 
verification and reference straightforward.

### Target Users:
Deep Research is particularly designed for professionals engaged in knowledge-intensive work, such as finance, 
science, and policy-making, as well as for consumers making detailed purchasing decisions.

### Performance:
The model has demonstrated high accuracy in internal evaluations, surpassing previous iterations in completing 
complex real-world tasks.

### Limitations:
While it significantly enhances research capabilities, it may occasionally produce inaccuracies (often referred to 
as "hallucinations"), which are more infrequent than in prior models. Additionally, it requires considerable 
computational resources for operation.

Overall, Deep Research represents a significant advancement toward achieving more autonomous AI systems capable of 
handling complex real-world tasks.

[
    ResponseFileSearchToolCall(
        id='fs_67dd3cef78a881929ed9ef3935dfa27d0e20091d0db0c021',
        queries=['What is deep research by OpenAI?'],
        status='completed',
        type='file_search_call',
        results=None
    ),
    ResponseOutputMessage(
        id='msg_67dd3cf2985c819299ee9b44d15e44de0e20091d0db0c021',
        content=[
            ResponseOutputText(
                annotations=[
                    AnnotationFileCitation(
                        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
                        index=1059,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
                        index=1214,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    )
                ],
                text='Deep Research by OpenAI is a new capability within ChatGPT aimed at performing extensive 
online research tasks with a high level of autonomy. \n\n### Key Features:\n- **Agentic Capability**: It can 
conduct multi-step research independently, synthesizing information from various sources to provide comprehensive 
and detailed reports.\n- **Efficiency**: What would typically take a human numerous hours can be accomplished in a 
fraction of that time—often within minutes.\n- **Advanced Reasoning**: Leveraging a version of the OpenAI o3 model 
optimized for web browsing and data analysis, it reasons through large volumes of text, images, and documents 
online to create accurate outputs.\n- **Documentation and Citations**: All outputs are fully documented with 
citations and detailed summaries, making verification and reference straightforward.\n\n### Target Users:\nDeep 
Research is particularly designed for professionals engaged in knowledge-intensive work, such as finance, science, 
and policy-making, as well as for consumers making detailed purchasing decisions.\n\n### Performance:\nThe model 
has demonstrated high accuracy in internal evaluations, surpassing previous iterations in completing complex 
real-world tasks.\n\n### Limitations:\nWhile it significantly enhances research capabilities, it may occasionally 
produce inaccuracies (often referred to as "hallucinations"), which are more infrequent than in prior models. 
Additionally, it requires considerable computational resources for operation.\n\nOverall, Deep Research represents 
a significant advancement toward achieving more autonomous AI systems capable of handling complex real-world 
tasks.',
                type='output_text'
            )
        ],
        role='assistant',
        status='completed',
        type='message'
    )
]

Customize the number of results you want to retrieve from the vector stores.

In [32]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        # You can see change in result only if you also provide include=["file_search_call.results"] 
        # then you will be able to see the difference
        "max_num_results": 2  
    }],
    include=["file_search_call.results"]
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output[0].results)
rich.print(response.output[1])

Deep Research is a new capability introduced by OpenAI within ChatGPT, designed to conduct multi-step research 
tasks on the internet. It allows users to perform complex research tasks that would normally take a human many 
hours, in just minutes. 

### Key Features of Deep Research:
1. **Reasoning and Synthesis**: It synthesizes large amounts of information from various online sources, similar to
the work of a research analyst.
2. **Independent Functionality**: Users provide a prompt, and Deep Research autonomously finds, analyzes, and 
synthesizes data.
3. **Optimized for Browsing and Analysis**: It uses a specialized version of the upcoming OpenAI o3 model, 
optimized for web browsing and data analysis.
4. **Documented Outputs**: The outputs are fully documented with citations, making it easy to verify the 
information.
5. **Wide Applications**: It caters to professionals in fields like finance, science, policy, and engineering, as 
well as general consumers seeking detailed product research.

### How It Works:
Deep Research uses reinforcement learning on real-world tasks and has been trained to handle complex inquiries 
through effective searching, interpreting, and backtracking as necessary. It reaches high performance on 
evaluations focused on real-world questions, demonstrating its advanced capabilities in finding niche and 
non-intuitive information.

[
    Result(
        attributes={},
        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
        filename='deep_research_blog.pdf',
        score=0.9811886993987157,
        text="Introducing deep research | OpenAI\n\n\nFebruary 2, 2025 Release\n\nIntroducing deep research\nAn 
agent that uses reasoning to synthesize large amounts of\nonline information and complete multi-step research 
tasks\nfor you. Available to Pro users today, Plus and Team next.\n\nTry on ChatGPT\n\nListen to article 8:18 
Share\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 
1/38\n\nhttps://openai.com/research/index/release/\nhttps://chatgpt.com/\nhttps://openai.com/\n\n\nToday we’re 
launching deep research in ChatGPT, a new agentic capability that\nconducts multi-step research on the internet for
complex tasks. It accomplishes in\ntens of minutes what would take a human many hours.\n\nDeep research is OpenAI's
next agent that can do work for you independently—you\ngive it a prompt, and ChatGPT will find, analyze, and 
synthesize hundreds of online\nsources to create a comprehensive report at the level of a research analyst. 
Powered\nby a version of the upcoming OpenAI o3 model that’s optimized for web browsing and\ndata analysis, it 
leverages reasoning to search, interpret, and analyze massive amounts\nof text, images, and PDFs on the internet, 
pivoting as needed in reaction to information\nit encounters.\n\nThe ability to synthesize knowledge is a 
prerequisite for creating new knowledge. For\nthis reason, deep research marks a significant step toward our 
broader goal of\ndeveloping AGI, which we have long envisioned as capable of producing novel\nscientific 
research.\n\nWhy we built deep research\n\nDeep research is built for people who do intensive knowledge work in 
areas like\nfinance, science, policy, and engineering and need thorough, precise, and reliable\nresearch. It can be
equally useful for discerning shoppers looking for hyper-\npersonalized recommendations on purchases that typically
require careful research,\nlike cars, appliances, and furniture. Every output is fully documented, with 
clear\ncitations and a summary of its thinking, making it easy to reference and verify the\ninformation. It is 
particularly effective at finding niche, non-intuitive information that\nwould require browsing numerous websites. 
Deep research frees up valuable time by\nallowing you to offload and expedite complex, time-intensive web research 
with just\none query.\n\nDeep research independently discovers, reasons about, and consolidates insights 
from\nacross the web. To accomplish this, it was trained on real-world tasks requiring\nbrowser and Python tool 
use, using the same reinforcement learning methods behind\nOpenAI o1, our first reasoning model. While o1 
demonstrates impressive capabilities in\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 2/38\n\nhttps://openai.com/\n\n\ncoding, math, and 
other technical domains, many real-world challenges demand\nextensive context and information gathering from 
diverse online sources. Deep\nresearch builds on these reasoning capabilities to bridge that gap, allowing it to 
take\non the types of problems people face in work and everyday life.\n\nHow to use deep research\n\nIn ChatGPT, 
select ‘deep research’ in the message composer and enter your query. Tell\nChatGPT what you need—whether it’s a 
competitive analysis on streaming platforms\nor a personalized report on the best commuter bike. You can attach 
files or\nspreadsheets to add context to your question. Once it starts running, a sidebar\nappears with a summary 
of the steps taken and sources used."
    ),
    Result(
        attributes={},
        file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
        filename='deep_research_blog.pdf',
        score=0.9372908482319836,
        text="Deep research\n\nHelp me find iOS and android adopt

ResponseOutputMessage(
    id='msg_67dd42afafbc8192bbcfaf367f7b1f5b0bd9fe3d8218592a',
    content=[
        ResponseOutputText(
            annotations=[
                AnnotationFileCitation(
                    file_id='file-8kai7amHc4aJ55jZ7Fx9xJ',
                    index=1371,
                    type='file_citation',
                    filename='deep_research_blog.pdf'
                )
            ],
            text='Deep Research is a new capability introduced by OpenAI within ChatGPT, designed to conduct 
multi-step research tasks on the internet. It allows users to perform complex research tasks that would normally 
take a human many hours, in just minutes. \n\n### Key Features of Deep Research:\n1. **Reasoning and Synthesis**: 
It synthesizes large amounts of information from various online sources, similar to the work of a research 
analyst.\n2. **Independent Functionality**: Users provide a prompt, and Deep Research autonomously finds, analyzes,
and synthesizes data.\n3. **Optimized for Browsing and Analysis**: It uses a specialized version of the upcoming 
OpenAI o3 model, optimized for web browsing and data analysis.\n4. **Documented Outputs**: The outputs are fully 
documented with citations, making it easy to verify the information.\n5. **Wide Applications**: It caters to 
professionals in fields like finance, science, policy, and engineering, as well as general consumers seeking 
detailed product research.\n\n### How It Works:\nDeep Research uses reinforcement learning on real-world tasks and 
has been trained to handle complex inquiries through effective searching, interpreting, and backtracking as 
necessary. It reaches high performance on evaluations focused on real-world questions, demonstrating its advanced 
capabilities in finding niche and non-intuitive information.',
            type='output_text'
        )
    ],
    role='assistant',
    status='completed',
    type='message'
)